### Additional Uses

In [1]:
import decimal

In [2]:
decimal.getcontext()

Context(prec=28, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])

In [3]:
decimal.getcontext().prec = 14

In [4]:
decimal.getcontext()

Context(prec=14, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])

In [5]:
decimal.getcontext().prec = 28

In [13]:
old_prec = decimal.getcontext().prec
decimal.getcontext().prec = 4
print(decimal.Decimal(1) / decimal.Decimal(3))
decimal.getcontext().prec = old_prec
print(decimal.Decimal(1) / decimal.Decimal(3))

0.3333
0.3333333333333333333333333333


In [14]:
class precision:
    def __init__(self, prec):
        self.prec = prec
        self.current_prec = decimal.getcontext().prec

    def __enter__(self):
        decimal.getcontext().prec = self.prec

    def __exit__(self, exc_type, exc_value, exc_tb):
        decimal.getcontext().prec = self.current_prec
        return False


In [15]:
with precision(3):
    print(decimal.Decimal(1) / decimal.Decimal(3))
print(decimal.Decimal(1) / decimal.Decimal(3))

0.333
0.3333333333333333333333333333


In [16]:
with decimal.localcontext() as ctx:
    ctx.prec = 3
    print(ctx.prec)

3


In [17]:
decimal.getcontext().prec

28

In [18]:
with decimal.localcontext() as ctx:
    ctx.prec = 3
    print(decimal.Decimal(1) / decimal.Decimal(3))
print(decimal.Decimal(1) / decimal.Decimal(3))

0.333
0.3333333333333333333333333333


In [19]:
from time import perf_counter, sleep

In [25]:
class Timer():
    def __init__(self):
        self.elapsed = 0

    def __enter__(self):
        self.start = perf_counter()
        return self

    def __exit__(self, exc_type, exc_value, exc_tb):
        self.stop = perf_counter()
        self.elapsed = self.stop - self.start
        return False

In [26]:
with Timer() as timer:
    sleep(1)

print(timer.elapsed)

1.0001369630000454


In [1]:
print('hello')

hello


In [29]:
import sys

class OutToFile:
    def __init__(self, fname):
        self._fname = fname
        self._current_stdout = sys.stdout

    def __enter__(self):
        self._file = open(self._fname, 'w')
        sys.stdout = self._file

    def __exit__(self, exc_type, exc_value, exc_tb):
        sys.stdout = self._current_stdout
        self._file.close()
        return False


In [6]:
with OutToFile('test.txt'):
    print(sys.stdout)
    print('Line1')
    print('Line2')

In [7]:
print('Line1')

Line1


In [8]:
sys.stdout

In [11]:
with open('test.txt') as f:
    print(f.readlines())

["<_io.TextIOWrapper name='test.txt' mode='w' encoding='UTF-8'>\n", 'Line1\n', 'Line2\n']


In [12]:
print('Line 1')

Line 1


In [16]:
class Tag:
    def __init__(self, tag):
        self._tag = tag

    def __enter__(self):
        print(f'<{self._tag}>', end='')


    def __exit__(self, exc_type, exc_value, exc_tb):
        print(f'</{self._tag}>', end='')

In [17]:
with Tag('p'):
    print('some ', end='')
    with Tag('b'):
        print('bold', end='')
    print(' text', end='')

<p>some <b>bold</b> text</p>

Title
- Item 1
    - sub item 1a
    - sub item 1b
- Item 2
    - sub item 2a
    - sub item 2b

In [22]:
class ListMaker:
    def __init__(self, title, prefix='- ', indent=3):
        self._title = title
        self._prefix = prefix
        self._indent = indent
        self._current_indent = 0
        print(title)

    def __enter__(self):
        self._current_indent += self._indent
        return self

    def __exit__(self, exc_type, exc_value, exc_tb):
        self._current_indent -= self._indent
        return False

    def print_item(self, arg):
        s = ' ' * self._current_indent + self._prefix + str(arg)
        print(s)


In [26]:
with ListMaker('Items') as lm:
    lm.print_item('Item 1')
    lm.print_item('Item 2')

Items
   - Item 1
   - Item 2


In [ ]:
with ListMaker('Items') as lm:
    lm.print_item('Item 1')
    with lm:
        lm.print_item('Sub Item 1a')
        lm.print_item('Sub Item 2b')
        with lm:
            lm.print_item('Sub Syb Item 2b - 1')
            lm.print_item('Sub Syb Item 2b - 2')
    lm.print_item('Item 2')
    with lm:
        lm.print_item('Sub Item 2a')
        lm.print_item('Sub Item 2b')

Items
   - Item 1
      - Sub Item 1a
      - Sub Item 2b
         - Sub Syb Item 2b - 1
         - Sub Syb Item 2b - 2
   - Item 2
      - Sub Item 2a
      - Sub Item 2b


In [30]:
with OutToFile('my_list.txt'):
    with ListMaker('Items') as lm:
        lm.print_item('Item 1')
        with lm:
            lm.print_item('Sub Item 1a')
            lm.print_item('Sub Item 2b')
            with lm:
                lm.print_item('Sub Syb Item 2b - 1')
                lm.print_item('Sub Syb Item 2b - 2')
        lm.print_item('Item 2')
        with lm:
            lm.print_item('Sub Item 2a')
            lm.print_item('Sub Item 2b')

In [31]:
print('test')

test


In [32]:
with open('my_list.txt') as f:
    for row in f:
        print(row, end='')

Items
   - Item 1
      - Sub Item 1a
      - Sub Item 2b
         - Sub Syb Item 2b - 1
         - Sub Syb Item 2b - 2
   - Item 2
      - Sub Item 2a
      - Sub Item 2b
